In [2]:
import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time # for sleep time
import csv
import re
import functools
import datetime #for csv file title

driver = webdriver.Chrome(r'C:\Users\hca.ITAM\Documents\chromedriver.exe')

driver.get('https://www.airbnb.mx/s/Whistler--Columbia-Brit%C3%A1nica--Canad%C3%A1/homes?refinement_paths%5B%5D=%2Fhomes&toddlers=0&query=Whistler%2C%20Columbia%20Brit%C3%A1nica%2C%20Canad%C3%A1&place_id=ChIJh3uVA7I8h1QR9s_1dehBt0o&search_type=filter_change&adults=1&s_tag=fxn88d36')

city = "Whistler"
now = datetime.datetime.now() #for csv file title

#For csv file names
def reduce_concat(x, sep=""):
    return functools.reduce(lambda x, y: str(x) + sep + str(y), x)

def paste(*lists, sep=" ", collapse=None):
    result = map(lambda x: reduce_concat(x, sep=sep), zip(*lists))
    if collapse is not None:
        return reduce_concat(result, sep=collapse)
    return list(result)

#Open CSV Writer
filename = paste(['airBnb_Whistler'], [now.year], ['-'], [now.month], ['-'], [now.day],['.csv'], sep='')
filename = ''.join(filename)
csv_file = open(filename, 'w', encoding='utf-8',newline = '')
writer = csv.writer(csv_file)

index = 1

house_all_links = []

while True:
    
    if index > 17:
        break
    try:
        print("Getting list of links from Page " + str(index))
        index = index + 1
        #driver.maximize_window()
        time.sleep(6) # give new page time to load
                
        house_list = (driver.find_elements_by_xpath('//a[@class="_1ol0z3h"]'))

        house_links = [house.get_attribute('href') for house in house_list ] 
        
        house_all_links.extend(house_links)
  
        # Locate the next button on the page.
        wait_button = WebDriverWait(driver, 10)
        
        next_button = wait_button.until(EC.element_to_be_clickable((By.XPATH,'//li[@class="_r4n1gzb"]')))
        
        next_button.click()

        
    except Exception as e:
        print(e)
        continue
        
print(house_all_links)

for url in house_all_links:

    # Initialize an empty dictionary for each review
    airbnb_dict = {}
    driver.get(url)

    print(url)
    try:
        time.sleep(1.5)
        try:
            house_title = driver.find_element_by_xpath('//h1/span[@class="_18hrqvin"]').text
        except:
            print("issue in house title")
            house_title = None

        try:
            house_type =  driver.find_element_by_xpath('//div[@class="_1p3joamp"]').text
        except:
            print("issue in house type")
            house_type = None
            
        time.sleep(3)    

        try:
            
            price = driver.find_element_by_xpath('//span/span[@class="_doc79r"]').text
        except:
            print("issue in price")
            price = None        

        try:
            user = driver.find_element_by_xpath('//div[@class="_8b6uza1"]').text
        except:
            print("issue in user")
            user = None  

        try:
            no_of_ratings = driver.find_element_by_xpath('//div[@class="_vy3ibx"]/h2/span').text

        except:
            print("issue in no_of_ratings")
            no_of_ratings = None  
        
        time.sleep(3.5)    
        try:
            rating = driver.find_element_by_xpath('//div[@class="_l0ao8q"]//div[@itemprop="ratingValue"]').get_attribute('content')

        except:
            print("issue in rating")
            rating = None

        time.sleep(2)        
        try:
            max_occupancy = driver.find_elements_by_xpath('//div/div[@class="_36rlri"]/div[@class="_czm8crp"]')

            house_detail = [detail.text for detail in max_occupancy ]

        except:
            print("Issue in occupncy")
            house_detail = None

        airbnb_dict['city'] = city
        airbnb_dict['house_title'] = house_title
        airbnb_dict['house_type'] = house_type
        airbnb_dict['price'] = price
        airbnb_dict['user'] = user
        airbnb_dict['no_of_reviews'] = no_of_ratings
        airbnb_dict['rating'] = rating
        airbnb_dict['house_detail'] = house_detail

        print("writing")
        writer.writerow(airbnb_dict.values())

        #print(i)
        print(house_title)
        print(house_type)
        print(price)
        print(user)
        print(no_of_ratings)
        print(rating)
        print(house_detail)

    except Exception as e:
        print(e)
        continue
        
csv_file.close()
driver.close()

Getting list of links from Page 1
Message: element click intercepted: Element <li class="_r4n1gzb">...</li> is not clickable at point (618, 692). Other element would receive the click: <div class="optanon-alert-box-wrapper  " role="alertdialog" aria-labelledby="alert-box-title" aria-describedby="alert-box-message" style="display: block; animation-name: slide-down-custom; animation-duration: 1000ms; animation-fill-mode: forwards; animation-timing-function: ease-out;">...</div>
  (Session info: chrome=75.0.3770.142)
  (Driver info: chromedriver=74.0.3729.6 (255758eccf3d244491b8a1317aa76e1ce10d57e9-refs/branch-heads/3729@{#29}),platform=Windows NT 10.0.17134 x86_64)

Getting list of links from Page 2
Message: element click intercepted: Element <li class="_r4n1gzb">...</li> is not clickable at point (618, 692). Other element would receive the click: <div class="optanon-alert-box-wrapper  " role="alertdialog" aria-labelledby="alert-box-title" aria-describedby="alert-box-message" style="disp

Message: element click intercepted: Element <li class="_r4n1gzb">...</li> is not clickable at point (618, 692). Other element would receive the click: <div class="optanon-alert-box-wrapper  " role="alertdialog" aria-labelledby="alert-box-title" aria-describedby="alert-box-message" style="display: block; animation-name: slide-down-custom; animation-duration: 1000ms; animation-fill-mode: forwards; animation-timing-function: ease-out;">...</div>
  (Session info: chrome=75.0.3770.142)
  (Driver info: chromedriver=74.0.3729.6 (255758eccf3d244491b8a1317aa76e1ce10d57e9-refs/branch-heads/3729@{#29}),platform=Windows NT 10.0.17134 x86_64)

Getting list of links from Page 15
Message: element click intercepted: Element <li class="_r4n1gzb">...</li> is not clickable at point (618, 692). Other element would receive the click: <div class="optanon-alert-box-wrapper  " role="alertdialog" aria-labelledby="alert-box-title" aria-describedby="alert-box-message" style="display: block; animation-name: slide

https://www.airbnb.mx/rooms/13348663?location=Whistler%2C%20Columbia%20Brit%C3%A1nica%2C%20Canad%C3%A1&toddlers=0&adults=1&previous_page_section_name=1000
writing
Peak View Penthouse
Loft entero
$1,880
Jane & James
158 evaluaciones
5
['4 huéspedes', '1 recámara', '1 cama', '1 baño', '\U000f0004\n158 evaluaciones', '\U000f0019\nVerificado']
https://www.airbnb.mx/rooms/9670429?location=Whistler%2C%20Columbia%20Brit%C3%A1nica%2C%20Canad%C3%A1&toddlers=0&adults=1&previous_page_section_name=1000
writing
⛰️ 1 Bdrm Modern Village Penthouse - view!
Departamento con servicios completo
$2,025
Victoria And Christian
427 evaluaciones
5
['4 huéspedes', '1 recámara', '2 camas', '1 baño', '\U000f0004\n1165 evaluaciones', '\U000f0019\nVerificado']
https://www.airbnb.mx/rooms/1295382?location=Whistler%2C%20Columbia%20Brit%C3%A1nica%2C%20Canad%C3%A1&toddlers=0&adults=1&previous_page_section_name=1000
writing
Whistler Creekside Villa - single nights now open!
Departamento entero
$1,151
Beth
333 evaluacio

In [ ]:
cols =["city","house_title","house_type","price","owner_name","no_of_reviews","rating","occupancy"]

#read in the csv file
airbnbdf = pd.read_csv('airBnb_Whistler2019-7-17.csv',encoding = 'utf-8', names=cols)
airbnbdf.head(5)

In [ ]:
#verify the datatypes
airbnbdf.dtypes

In [ ]:
airbnbdf.price = airbnbdf.price.astype(str)
airbnbdf.no_of_reviews = airbnbdf.no_of_reviews.astype(str)


def remove_special_chars(val):
    new_val = re.findall('[0-9]',val)
    return "".join(new_val)

airbnbdf.price = airbnbdf.price.apply(remove_special_chars)
print(airbnbdf.price.head(5))


In [ ]:

airbnbdf.no_of_reviews = airbnbdf.no_of_reviews.apply(remove_special_chars)
print(airbnbdf.no_of_reviews.head(5))


In [ ]:
def split_to_list(str_val):
    return str_val.split(",")
    

airbnbdf['occupancy_lst'] = airbnbdf.occupancy.apply(split_to_list)

airbnbdf.occupancy_lst.sample(5)

In [ ]:
len(airbnbdf)

airbnbdf.occupancy_lst[0][0]


In [ ]:
airbnbdf = airbnbdf.assign(no_of_guest=airbnbdf.price.mean(), house_size=airbnbdf.price.mean(), no_of_beds = airbnbdf.price.mean(), no_of_baths=airbnbdf.price.mean())


# one time thing, however this is not a nice way to do this
# I learnt using assign function would do it, which I have incorporated in my code later, 
#for eg: airbnbdf.assign(beds=airbnbdf.no_of_beds.apply(remove_chars))

for i in range(len(airbnbdf)):
    try:

        airbnbdf.no_of_guest[i] = airbnbdf.occupancy_lst[i][0]
        airbnbdf.house_size[i]  = airbnbdf.occupancy_lst[i][1]
        airbnbdf.no_of_beds[i]  = airbnbdf.occupancy_lst[i][2]
        airbnbdf.no_of_baths[i]  = airbnbdf.occupancy_lst[i][3]
        
    except:
        continue

In [ ]:
airbnbdf.no_of_guest = airbnbdf.no_of_guest.apply(remove_special_chars)
print(airbnbdf.no_of_guest.head(5))

In [ ]:
print(sum(airbnbdf.house_size.isna()))

print(sum(airbnbdf.house_size.isnull()))

In [ ]:
def convert_chars(val):
   ##studio is tagged as 0.5 bedroom
    if val == " 'Studio'":
        new_val = '0.5'
    else:
        new_val = re.findall('[0-9]',val)
    
    return "".join(new_val)

airbnbdf.house_size = airbnbdf.house_size.astype(str)
airbnbdf = airbnbdf.assign(rooms=airbnbdf.house_size.apply(convert_chars))

airbnbdf.sample(14)

In [ ]:
def remove_chars(val):
    new_val = re.findall('[0-9.]+',val)
    return "".join(new_val)


airbnbdf.no_of_beds = airbnbdf.no_of_beds.astype(str)
airbnbdf.no_of_baths = airbnbdf.no_of_baths.astype(str)

airbnbdf = airbnbdf.assign(beds=airbnbdf.no_of_beds.apply(remove_chars))
airbnbdf = airbnbdf.assign(bath=airbnbdf.no_of_baths.apply(remove_chars))
airbnbdf.sample(4)

In [ ]:
airbnbdf.dropna() #check 

In [ ]:
airbnbdfClean = airbnbdf.dropna() #drop all NaN's

In [ ]:
#convert price to float
airbnbdfClean.price=airbnbdfClean.price.astype(float)

#convert no_of_reviews to float
airbnbdfClean.no_of_reviews=airbnbdfClean.no_of_reviews.astype(int)

#convert no_of_guest to float
airbnbdfClean.no_of_guest=airbnbdfClean.no_of_guest.astype(float)

#convert house to float
airbnbdfClean.rooms=airbnbdfClean.rooms.astype(float)

#convert house to float
airbnbdfClean.beds=airbnbdfClean.beds.astype(float)

#convert house to float
airbnbdfClean.bath=airbnbdfClean.bath.astype(float)

In [ ]:
airbnbdfClean.sample(5)

In [ ]:
# write the cleaned and massaged data to csv for future analysis

airbnbdfClean.to_csv('airbnbdfClean.csv', index=False)